# evaluomeR - RSKC - metric relevancy

In [1]:
library("sparcl")
library("evaluomeR")

options(scipen=10)

Loading required package: SummarizedExperiment
Loading required package: GenomicRanges
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: 'BiocGenerics'

The following objects are masked from 'package:parallel':

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs

The following objects are masked from 'package:base':

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which

# Table of contents
* [Dataset](#dataset)
* [evaluomeR CBIs](#cbis)
* [Metrics relevancy](#metrics_relevancy)
    * [Optimal K value](#optimal_k)
    * [Figuring out the L<sub>1</sub> upper boundry](#l1_boundry)
    * [Relevancy table](#relevancy_table)

# Dataset <a class="anchor" id="dataset"></a>

In [2]:
seed = 13606
set.seed(seed)

agro_df = read.csv(paste0(getwd(), "/","data/agro.csv"), header=TRUE, stringsAsFactors=FALSE)
head(agro_df)

Description,ANOnto,AROnto,CBOOnto,CBOOnto2,CROnto,DITOnto,INROnto,LCOMOnto,NACOnto,NOCOnto,NOMOnto,POnto,PROnto,RFCOnto,RROnto,TMOnto,TMOnto2,WMCOnto,WMCOnto2
ADO,0.0000000,3.9503849,0.9991446,0.9991446,0.9957228,3,0.9991446,1.999142,1.000000,292.000000,2.9632164,0.9957228,0.7478411,3.9623610,0.252158895,0.00000000,0.000000,1.999142,1.000000
AEO,0.9298246,0.5438596,0.9824561,0.9824561,0.0000000,5,0.9824561,2.357143,1.000000,3.733333,0.9824561,0.5789474,0.5000000,1.9649123,0.500000000,0.00000000,0.000000,2.357143,1.000000
AFO,0.7500000,0.0000000,0.8750000,0.8750000,3998.8750000,3,0.8750000,1.333333,1.000000,3.500000,2275.7500000,0.2500000,0.9996157,2276.6250000,0.000384341,0.00000000,0.000000,1.333333,1.000000
AGRO,0.9907407,3.1018519,1.0694444,1.0694444,0.3634259,16,1.0694444,7.695971,1.052174,2.287129,1.2037037,1.0555556,0.5295316,2.2731481,0.470468432,0.06264501,2.148148,9.134783,1.186957
AGRORDF,1.2362637,0.0000000,1.0659341,1.0659341,0.0000000,6,1.0329670,2.467532,1.077465,4.700000,0.5879121,0.8571429,0.3627119,1.6538462,0.637288136,0.07182320,2.000000,2.676056,1.084507
ANAEETHES,0.0000000,0.0000000,0.6666667,0.6666667,1107.6666670,2,0.6666667,1.000000,1.000000,2.000000,0.0000000,0.0000000,0.0000000,0.6666667,1.000000000,0.00000000,0.000000,1.000000,1.000000


# evaluomeR CBIs <a class="anchor" id="cbis"></a>
Available clusterboot interfaces (clustering methods) in evaluomeR

In [3]:
cat(paste(shQuote(evaluomeRSupportedCBI(), type="cmd"), collapse=", "))

"kmeans", "clara", "clara_pam", "hclust", "pamk", "pamk_pam", "rskc"

# Metrics relevancy <a class="anchor" id="metrics_relevancy"></a>
We need to compute the optimal K value of the dataset and the L1 boundry in order to compute the table of metrics relevancy.

## Optimal K value <a class="anchor" id="optimal_k"></a>
Calculating the optimal K value with *kmeans* CBI. Here, we make use of *evaluomeR* to figure out the optimal $k$ value. We consider the $k$ range [3,6] for the analysis, avoiding $k=2$ to prevent from having binary classifications.

In [4]:
k.range=c(3,6)
cbi = "kmeans"

stab_range = stabilityRange(data=agro_df, k.range=k.range, 
                            bs=100, seed=seed,
                            all_metrics=TRUE,
                            cbi=cbi)
stab = standardizeStabilityData(stab_range)

# Qual
qual_range = qualityRange(data=agro_df, k.range=k.range, 
                            all_metrics=TRUE, seed=seed,
                            cbi=cbi)
qual = standardizeQualityData(qual_range)

# K opt
k_opt = getOptimalKValue(stab_range, qual_range, k.range= k.range)
optimal_k = as.numeric(k_opt$Global_optimal_k)


Data loaded.
Number of rows: 78
Number of columns: 20


Processing all metrics, 'merge', in dataframe (19)
	Calculation of k = 3
	Calculation of k = 4
	Calculation of k = 5
	Calculation of k = 6

Data loaded.
Number of rows: 78
Number of columns: 20


Processing all metrics, 'merge', in dataframe (19)
	Calculation of k = 3
	Calculation of k = 4
	Calculation of k = 5
	Calculation of k = 6
Processing metric: all_metrics

	Maximum stability and quality values matches the same K value: '3'



We are going to use $k=3$ as the optimal K value.

## Figuring out the L<sub>1</sub> upper boundry <a class="anchor" id="l1_boundry"></a>

This algorithm for tuning the L<sub>1</sub> parameter and others are presented in 'sparcl' R package.
Considering that for the dataset the global optimal $k$ is $k=3$, we can now compute the permutations to figure out the boundry L<sub>1</sub> with the method 'KMeansSparseCluster.permute' from 'sparcl'.

Note: 1 $<$ L<sub>1</sub> $\leq$ $\sqrt{num.variables}$.

In [5]:
agro_data = agro_df[-1] # Removing 'Description' column as it is not numeric
wbounds = seq(2,sqrt(ncol(agro_data)), len=30)
km.perm <- sparcl::KMeansSparseCluster.permute(agro_data, K=optimal_k, wbounds=wbounds, nperms=5, silent=TRUE)
L1 = km.perm$bestw
cat(paste0("Best L1 for k=", optimal_k, " is: ", L1))

Best L1 for k=3 is: 2

The best L<sub>1</sub> upper boundry for $k=3$ is $L_{1}=2$.

## Relevancy table <a class="anchor" id="relevancy_table"></a>

Now we know that optimal $k$ value is $k=3$ and that $L_{1}=2$, from our previous analysis. With this we have everything set up to get the relevancy table of the metrics.

**Note**: Remove the first column `Description` method `evaluomeR::getMetricsRelevancy`, that is why we use `agro_data` instead of `agro_df`.

In [6]:
agro_relevancy = getMetricsRelevancy(agro_data, alpha=0, k=optimal_k, L1=L1, seed=seed)
relevancy_table = agro_relevancy$relevancy
relevancy_table

[1] "Alpha set as: 0"
[1] "L1 set as: 2"


,metric,weight
5,CROnto,0.999994852839666715
11,NOMOnto,0.002268831192932223
14,RFCOnto,0.002268633768701414
10,NOCOnto,0.000000093242941073
18,WMCOnto,0.000000060450218928
6,DITOnto,0.000000047918994503
8,LCOMOnto,0.000000010651039836
2,AROnto,0.000000001000350998
17,TMOnto2,0.000000000945103542
12,POnto,0.000000000393327390
